In [1]:
#BRUTE FORCE APPROACH
import time
from multiprocessing import Pool
popular_pins = ["123456", "654321", "111111", "000000", "999999", "777777", "101010", "123123", "112233", "321654", "123321", "520131", "520520", "147258", "123654", "666666", "121212", "789456",
                "159753", "030379", "101471"]
pattern_pins = ["112233", "121212", "111222", "123321", "654321", "010119", "199519", "010120"]
max_attempts = 10
time_limit = 300
common_2fa_codes = ["123456", "654321", "111111", "000000", "999999","101010", "123123", "112233", "321654", "123321", "030379", "010120", "020220", "041519", "121212"]
def mock_2fa_verification(user_input_code):
    return user_input_code in common_2fa_codes
def try_pin(pin, target_pin):
    return pin == target_pin
def attempt_pins(pins_subset, target_pin, attempts, user_2fa_code):
    for pin in pins_subset:
        if attempts['count'] >= max_attempts:
            break
        if try_pin(pin, target_pin) and mock_2fa_verification(user_2fa_code):
            return pin
        attempts['count'] += 1
    return None
def brute_force_optimized(target_pin, attempts, user_2fa_code):
    for pin in popular_pins:
        if attempts['count'] >= max_attempts:
            break
        if try_pin(pin, target_pin) and mock_2fa_verification(user_2fa_code):
            return pin
    for pin in pattern_pins:
        if attempts['count'] >= max_attempts:
            break
        if try_pin(pin, target_pin) and mock_2fa_verification(user_2fa_code):
            return pin
    return None
def smart_guessing(target_pin, attempts, user_2fa_code):
    likely_pins = ["199519", "010120", "031579", "071219"]
    for pin in likely_pins:
        if attempts['count'] >= max_attempts:
            break
        if try_pin(pin, target_pin) and mock_2fa_verification(user_2fa_code):
            return pin
    return None
def parallel_brute_force(all_possible_pins, target_pin, attempts, user_2fa_code):
    num_workers = 2
    chunk_size = len(all_possible_pins) // num_workers
    pin_chunks = [all_possible_pins[i:i + chunk_size] for i in range(0, len(all_possible_pins), chunk_size)]
    with Pool(num_workers) as pool:
        results = pool.starmap(attempt_pins, [(chunk, target_pin, attempts, user_2fa_code) for chunk in pin_chunks])
    for result in results:
        if result:
            return result
    return None
def run_pin_cracking_model(target_pin, user_2fa_code):
    attempts = {'count': 0}
    print(f"Attempting to crack PIN: {target_pin}")
    found_pin = brute_force_optimized(target_pin, attempts, user_2fa_code)
    if found_pin:
        print(f"PIN found: {found_pin} for target PIN: {target_pin}")
        return
    found_pin = smart_guessing(target_pin, attempts, user_2fa_code)
    if found_pin:
        print(f"PIN found: {found_pin} for target PIN: {target_pin}")
        return
    all_possible_pins = [str(i).zfill(6) for i in range(1000000)]
    found_pin = parallel_brute_force(all_possible_pins, target_pin, attempts, user_2fa_code)
    if found_pin:
        print(f"PIN found: {found_pin} for target PIN: {target_pin}")
    else:
        print(f"Failed to crack the PIN: {target_pin} within {max_attempts} attempts.")
if __name__ == "__main__":
    target_pin = input("Enter the target 6-digit PIN to crack: ")
    user_2fa_code = input("Enter the 2FA code: ")
    run_pin_cracking_model(target_pin, user_2fa_code)

Enter the target 6-digit PIN to crack: 789456
Enter the 2FA code: 123456
Attempting to crack PIN: 789456
PIN found: 789456 for target PIN: 789456


In [6]:
#BRUTE FORCE APPROACH WITH TIMING ATTACK(SIDE-CHANNEL ATTACK)
import time
from multiprocessing import Pool
popular_pins = ["123456", "654321", "111111", "000000", "999999", "777777", "101010", "123123", "112233", "321654", "123321", "520131", "520520", "147258", "123654", "666666", "121212", "789456",
                "159753", "030379", "101471"]
pattern_pins = ["112233", "121212", "111222", "123321", "654321", "010119", "199519", "010120"]
max_attempts = 10
time_limit = 300
common_2fa_codes = ["123456", "654321", "111111", "000000", "999999","101010", "123123", "112233", "321654", "123321", "030379", "010120", "020220", "041519", "121212"]
def mock_2fa_verification(user_input_code):
    return user_input_code in common_2fa_codes
def timing_attack_simulation(pin_guess, target_pin):
    time.sleep(0.01)
    for i in range(len(pin_guess)):
        if pin_guess[i] != target_pin[i]:
            return False, time.perf_counter()
        time.sleep(0.005)
    return True, time.perf_counter()
def try_pin(pin, target_pin):
    start_time = time.perf_counter()
    valid, end_time = timing_attack_simulation(pin, target_pin)
    response_time = end_time - start_time
    return valid, response_time
def attempt_pins(pins_subset, target_pin, attempts, user_2fa_code):
    for pin in pins_subset:
        if attempts['count'] >= max_attempts:
            break
        valid, response_time = try_pin(pin, target_pin)
        if valid and mock_2fa_verification(user_2fa_code):
            return pin
        attempts['count'] += 1
    return None
def brute_force_optimized(target_pin, attempts, user_2fa_code):
    for pin in popular_pins:
        if attempts['count'] >= max_attempts:
            break
        valid, response_time = try_pin(pin, target_pin)
        if valid and mock_2fa_verification(user_2fa_code):
            return pin
    for pin in pattern_pins:
        if attempts['count'] >= max_attempts:
            break
        valid, response_time = try_pin(pin, target_pin)
        if valid and mock_2fa_verification(user_2fa_code):
            return pin
    return None
def smart_guessing(target_pin, attempts, user_2fa_code):
    likely_pins = ["199519", "010120", "031579", "071219"]
    for pin in likely_pins:
        if attempts['count'] >= max_attempts:
            break
        valid, response_time = try_pin(pin, target_pin)
        if valid and mock_2fa_verification(user_2fa_code):
            return pin
    return None
def parallel_brute_force(all_possible_pins, target_pin, attempts, user_2fa_code):
    num_workers = 2
    chunk_size = len(all_possible_pins) // num_workers
    pin_chunks = [all_possible_pins[i:i + chunk_size] for i in range(0, len(all_possible_pins), chunk_size)]
    with Pool(num_workers) as pool:
        results = pool.starmap(attempt_pins, [(chunk, target_pin, attempts, user_2fa_code) for chunk in pin_chunks])
    for result in results:
        if result:
            return result
    return None
def run_pin_cracking_model(target_pin, user_2fa_code):
    attempts = {'count': 0}
    print(f"Attempting to crack PIN: {target_pin}")
    found_pin = brute_force_optimized(target_pin, attempts, user_2fa_code)
    if found_pin:
        print(f"PIN found: {found_pin} for target PIN: {target_pin}")
        return
    found_pin = smart_guessing(target_pin, attempts, user_2fa_code)
    if found_pin:
        print(f"PIN found: {found_pin} for target PIN: {target_pin}")
        return
    all_possible_pins = [str(i).zfill(6) for i in range(1000000)]
    found_pin = parallel_brute_force(all_possible_pins, target_pin, attempts, user_2fa_code)
    if found_pin:
        print(f"PIN found: {found_pin} for target PIN: {target_pin}")
    else:
        print(f"Failed to crack the PIN: {target_pin} within {max_attempts} attempts.")
if __name__ == "__main__":
    target_pin = input("Enter the target 6-digit PIN to crack: ")
    user_2fa_code = input("Enter the 2FA code: ")
    run_pin_cracking_model(target_pin, user_2fa_code)

Enter the target 6-digit PIN to crack: 123456
Enter the 2FA code: 121212
Attempting to crack PIN: 123456
PIN found: 123456 for target PIN: 123456
